<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Basic_Networking_Tools/SSH_Tunneling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__SSH Tunneling__*

In the last section, we built a tool that allowed us to run commands by entering them into an SSH client on a remote SSH server. Another technique would be to use an __SSH Tunnel__. Instead of sending commands to the server, an SSH tunnel would send network traffic packaged inside of SSH, and the SSH server would unpackage and deliver it.

Imagine that you're in the following situation: You have remote access to an SSH server on an internal network, but you want access to the web server on the same network. You can't access the web server directly. The server with SSH installed does have access, but this SSH server doesn't have the tools you want to use.

One way to overcome this problem is to set up a __Forward SSH Tunnel__. This would allow you to, for example, run the command:
```
ssh -L 8008:web:80 rick@sshserver
```
to connect to the SSH server as the user __rick__ and set up port __8008__ on your local system. Anything you send to port 8008 will travel down the existing SSH tunnel to the SSH server, which would deliver it to the web server. __Figure 2-1__ shows this in action.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C2-Basic_Networking_Tools/Client_to_Server.png?raw=true" alt="From Client to Server" width="40%">
<p style="text-align:center"><strong><em>Figure 2-1</strong></em></p>
</div>

That's pretty cool, but recall that not many Winzozz systems are running an SSH server service. Not all is lost, though. We can configure a __Reverse SSH Tunneling__ connection. In this case, we connect to our own SSH server from the Winzozz client in the usual fashion. Through that SSH connection, we also specify a remote port on the SSH server that gets tunneled to the local host and port, as shown in __Fugure 2-2__. We could use this local host and port, for example, to expose port 3389 to access an internal system using __Remote Desktop__ or to access another system that the Winzozz client can access (like the web server in our example).

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C2-Basic_Networking_Tools/Server_to_Client.png?raw=true" alt="From Client to Server" width="40%">
<p style="text-align:center"><strong><em>Figure 2-2</strong></em></p>
</div>

The paramiko demo files include a file called __rforward.py__ that does exactly this. It works perfectly as is, so we won't reprint that file in this book. We will, however, point out a couple of important points and run through an example of how to use it. Open __rforward.py__, skip to __main()__, and follow along:

In [ ]:
def main():
    options, server, remote = parse_options() #[1]
    password = None
    if options.readpass:
        password = getpass.getpass('Enter SSH password: ')
    client = paramiko.SSHClient() #[2]
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.WarningPolicy())
    
    verbose('Connecting to ssh host %s:%d ...' % (server[0], server[1]))
    try:
        client.connect(
            server[0],
            server[1],
            key_filename=options.keyfile,
            look_for_keys=options.look_for_keys,
            password=password
        )
    except Exception as e:
        print('*** Failed to connect to %s:%d: %r' % (server[0], server[1], e))
        sys.exit(1)
    
    verbose(
        'Now forwarding remote port %d to %s: %d ...'
        % (options.port, remote[0], remote[1])
    )

    try:
        reverse_forward_tunnel( #[3]
            options.port, remote[0], remote[1], client.get_transport()
        )
    except KeyboardInterrupt:
        print('C-c: Port forwarding stopped.')
        sys.exit(0)

The few lines at the top __[1]__ double-check to make sure all the necessary arguments are passed to the script before setting up the Paramiko SSH client connection __[2]__ (which should look very familiar). The final section in __main()__ calls the __reverse_forward_tunnel__ function __[3]__.

Let's have a look at that function:

In [ ]:
def reverse_forward_tunnel(server_port, remote_host, remote_port, transport):
    transport.request_port_forward('', server_port) #[1]
    while True:
        chan = transport.accept(1000) #[2]
        if chan is None:
            continue
        thr = threading.Thread( #[3]
            target=handler, args=(chan, remote_host, remote_port)
        )

        thr.setDaemon(True)
        thr.start()

In Paramiko, there are two main communication methods: __transport__, which is responsible for making and maintaining the encrypted connection, and __channel__, which acts like a socket for sending and receiving data over the encrypted transport session. Here we start to use Paramiko's __request_port_forward__ to forward TCP connections from a port __[1]__ on the SSH server and start up a new transport channel __[2]__. Then, over the channel, we call the function handler __[3]__.

But we're not done yet. We need to code the __handler__ function to manage the communication for each thread:

In [ ]:
def handler(chan, host, port):
    sock = socket.socket()
    try:
        sock.connect((host, port))
    except Exception as e:
        verbose('Forwarding request to %s:%d failed: %r' % (host, port, e))
        return

    verbose(
        'Connected! Tunnel open %r -> %r -> %r'
        % (chan.origin_addr, chan.getpeername(), (host, port))
    )
    while True: #[1]
        r, w, x = select.select([sock, chan], [], [])
        if sock in r:
            data = sock.recv(1024)
            if len(data) == 0:
                break
            chan.send(data)
        if chan in r:
            data = chan.recv(1024)
            if len(data == 0:
                break
            sock.send(data))
    chan.close()
    sock.close()
    verbose('Tunnel closed from %r' % (chan.origin_addr,))

And finally, the data is sent and received __[1]__. We give it a try in the next section.

# *__Kicking the tires__*

We'll run __rforward.py__ from our Winzozz system and configure it to be the middleman as we tunnel traffic from a web server to our Kali SSH server:
```
C:\Users\rick> python rforward.py 192.168.1.203 -p 8081 -r 192.168.1.207:3000 --user=rick --password
Enter SSH password:
Connecting to ssh host 192.168.1.203:22 ...
Now forwarding remote port 8081 to 192.168.1.207:3000 ...
```
You can see that on the Winzozz machine, we made a connection to the SSH server at 192.168.1.203 and opened port 8081 on that server, which will forward traffic to 102.168.1.207 port 30000. Now if we browse to http://127.0.1:8081 on our Linux server, we connect to the web server at 192.168.1.207:3000 through the SSH tunnel. If you flip back to the Winzozz machine, you can also see the connection being made in Paramiko:
```
Connected! Tunnel open ('127.0.0.1', 54690) -> ('192.168.1.203', 22) -> ('192.168.1.207', 3000)
```
SSH and SSH tunneling are important concepts to understand and use. Black hats should know when and exactly how to use SSH and SSH tunneling, and Paramiko makes it possible to add SSH capabilities to your existing Python tools.

We've created some very simple yet very useful tools in this chapter. We encourage you to expand and modify them as necessary to develop a firm grasp on Python's networking features. You could use these tools during penetrations tests, post-exploitation, or bug hunting. Let's move on to using raw sockets and performing network sniffing. Then we'll combine the two to create a pure Python host discovery scanner.

